<a href="https://colab.research.google.com/github/Jonnybobba/NBA_Analysis/blob/main/NBAData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-2.4.7'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:15 http://archive.ubuntu.com/ubuntu b

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-02-20 19:29:17--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-02-20 19:29:17 (6.02 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://nbadatabasebucket.s3.amazonaws.com/Team_00-20.csv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("Team_00-20.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(10)

+----+--------------------+---+-----+----+----+-----+---+----+-----+----+----+-----+----+----+-----+----+----+----+----+----+---+----+----+-----+--------+------------+
|Year|                Team|  G|   MP|  FG| FGA|  FG%| 3P| 3PA|  3P%|  2P| 2PA|  2P%|  FT| FTA|  FT%| ORB| DRB| TRB| AST| STL|BLK| TOV|  PF|  PTS|Playoffs|Championship|
+----+--------------------+---+-----+----+----+-----+---+----+-----+----+----+-----+----+----+-----+----+----+----+----+----+---+----+----+-----+--------+------------+
|2000|    Sacramento Kings| 82|20080|39.7|88.2|0.449|6.1|17.1|0.354|33.6|71.1|0.472|20.3|26.3|0.771|12.5|34.2|46.7|23.4|10.0|5.5|15.5|20.2|105.6|    true|       false|
|2000|     Milwaukee Bucks| 82|19780|41.0|89.5|0.458|7.4|19.5|0.379|33.6|70.0| 0.48|19.4|24.7|0.787|12.8|32.9|45.8|24.3| 8.8|5.1|14.8|25.4|108.8|    true|       false|
|2000|  Los Angeles Lakers| 82|19905|40.9|87.9|0.465|5.8|16.8|0.344|35.1|71.1|0.494|20.9|30.7|0.683|14.3|33.9|48.2|24.8| 7.4|6.4|15.6|24.6|108.4|    true|      

In [6]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://nbadatabasebucket.s3.amazonaws.com/Opp_00-20.csv"
spark.sparkContext.addFile(url)

df2 = spark.read.option('header', 'true').csv(SparkFiles.get("Opp_00-20.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df2.show(10)

+----+--------------------+---+-----+----+----+-----+---+----+-----+----+----+-----+----+----+-----+----+----+----+----+---+---+----+----+-----+---------+------------+
|Year|                Team|  G|   MP|  FG| FGA|  FG%| 3P| 3PA|  3P%|  2P| 2PA|  2P%|  FT| FTA|  FT%| ORB| DRB| TRB| AST|STL|BLK| TOV|  PF|  PTS|Playoffs2|Championship|
+----+--------------------+---+-----+----+----+-----+---+----+-----+----+----+-----+----+----+-----+----+----+----+----+---+---+----+----+-----+---------+------------+
|2000|     New York Knicks| 82|19905|35.7|85.7|0.417|6.2|17.6|0.352|29.5|68.1|0.434|20.6|28.0|0.733|12.1|33.6|45.8|21.9|8.3|4.7|16.9|24.4| 98.2|     true|       false|
|2000|          Miami Heat| 82|19880|37.4|86.8|0.431|5.0|15.1|0.331|32.5|71.7|0.452|18.6|25.2|0.737|12.7|35.0|47.7|19.7|8.3|5.9|17.7|25.3| 98.5|     true|       false|
|2000|   San Antonio Spurs| 82|19830|38.3|91.5|0.419|4.6|14.1|0.329|33.7|77.4|0.435|16.7|22.5|0.741|13.1|32.9|45.9|21.8|8.1|5.8|15.0|26.3| 98.0|     true|      

In [7]:
import pandas as pd

In [ ]:
Team_DF = df.toPandas()
print(Team_DF)